In [1]:
import os
import sys
import yaml
from typing import (
    Dict,
    List, 
    Optional,
    Tuple,
    Union
)

In [2]:
__file__ = os.path.join(os.getcwd(),"cs_wrap1.ipynb")

In [3]:
mod_path = os.path.join(os.path.dirname(os.path.abspath(__file__)),"..")

In [4]:
sys.path.append(mod_path)

In [27]:
from convert_source.cs_utils.bids_info import (
    construct_bids_dict,
    construct_bids_name,
    search_bids
)

In [28]:
from convert_source.cs_utils.utils import (
    BIDSimg,
    collect_info,
    convert_image_data,
    dict_multi_update,
    get_metadata,
    list_in_substr,
    SubInfoError,
    SubDataInfo,
    read_json,
    # new functions
    comp_dict,
    depth,
    list_dict
)

In [8]:
from convert_source.cs_utils.fileio import(
    LogFile,
    TmpDir
)

In [9]:
from convert_source.cs_utils.const import DEFAULT_CONFIG

In [10]:
from convert_source.imgio.dcmio import (
    is_valid_dcm
)

In [18]:
# test_config:
test_config = os.path.abspath(
    os.path.join(
        __file__,
        '..',
        '..',
        'notebooks.old',
        'config.test',
        'config.default.yml'
        ))
test_config

'/Users/adebayobraimah/Desktop/projects/convert_source/notebooks.old/config.test/config.default.yml'

In [19]:
def read_config(config_file: Optional[str] = "", 
                verbose: Optional[bool] = False
                ) -> Tuple[Dict[str,str],Dict,Dict,Dict,List[str]]:
    '''Reads configuration file and creates a dictionary of search terms for 
    each modality provided that each BIDS modality is used as a key via the 
    keyword 'modality_search'. Should BIDS related parameter descriptions need 
    to be used when renaming files, the related search and mapping terms can included 
    via the keywords 'bids_search' and 'bids_map', respectively. If these keywords 
    are not specified, then empty dictionaries are returned. Should exclusions be provided 
    (via the key 'exclude') then an exclusion list is created. Should this not be provided, 
    then an empty list is returned.

    BIDS modalities:
        - anat:
            - T1w, T2w, FLAIR, etc.
        - func:
            - bold
                - task:
                    - resting state, <task-name>
        - dwi
        - fmap

    Usage example:
        >>> [search_dict, bids_search, bids_map, meta_dict, exclusion_list] = read_config(config_file)
    
    Arguments:
        config_file: File path to yaml configuration file. If no file is used, then the default configuration file is used.
        verbose: Prints additional information to screen.
    
    Returns: 
        Tuple of dictionaries and a list that consists of:
            * search_dict: Nested dictionary of heuristic modality search terms for BIDS modalities.
            * bids_search: Nested dictionary of heuristic BIDS search terms.
            * bids_map: Corresponding nested dictionary of BIDS mapping terms to rename files to.
            * meta_dict: Nested dictionary of metadata terms to write to JSON file(s).
            * exclusion_list: List of exclusion terms.
    
    Raises:
        ConfigFileReadError: Error that arises if no heuristic search terms are provided.
    '''
    class ConfigFileReadError(Exception):
        pass

    if config_file:
        config_file: str = os.path.abspath(config_file)
    else:
        config_file: str = DEFAULT_CONFIG

    with open(config_file) as file:
        data_map: Dict[str,str] = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
    
    # Required modality search terms
    if any("modality_search" in data_map for element in data_map):
        if verbose:
            print("Categorizing search terms")
        search_dict: Dict[str,str] = data_map["modality_search"]
        del data_map["modality_search"]
    else:
        if verbose:
            print("Heuristic search terms required. Exiting...")
        raise ConfigFileReadError("Heuristic search terms required. Exiting...")
    
    # BIDS search terms
    if any("bids_search" in data_map for element in data_map):
        if verbose:
            print("Including BIDS related search term settings")
        bids_search: Dict[str,str] = data_map["bids_search"]
        del data_map["bids_search"]
    else:
        if verbose:
            print("No BIDS related search term settings")
        meta_dict: Dict = dict()
    
    # BIDS mapping terms
    if any("bids_map" in data_map for element in data_map):
        if verbose:
            print("Corresponding BIDS mapping settings")
        bids_map: Dict[str,str] = data_map["bids_map"]
        del data_map["bids_map"]
    else:
        if verbose:
            print("No BIDS mapping settings")
        meta_dict: Dict = dict()
    
    # Metadata terms
    if any("metadata" in data_map for element in data_map):
        if verbose:
            print("Including additional settings for metadata")
        meta_dict: Dict[str,Union[str,int]] = data_map["metadata"]
        del data_map["metadata"]
    else:
        if verbose:
            print("No metadata settings")
        meta_dict: Dict = dict()
    
    # Exclusion terms  
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("Exclusion option implemented")
        exclusion_list: List[str] = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("Exclusion option not implemented")
        exclusion_list: List = list()
        
    return (search_dict,
            bids_search,
            bids_map,
            meta_dict,
            exclusion_list)

In [20]:
[search_dict,bids_search,bids_map,meta_dict,exclusion_list] = read_config(config_file=test_config)

In [26]:
# search_dict
# bids_search
# bids_map
# meta_dict
# exclusion_list

In [32]:
# Check BIDS search and map dicts
comp_dict(d1=bids_search,d2=bids_map)

True

In [ ]:
# check depth of dicts

In [37]:
depth(search_dict['func'])

3

In [38]:
depth(search_dict['anat'])

2

In [29]:
par_img_dir = "test.img"

In [30]:
par_img_dir = os.path.abspath(par_img_dir)

In [31]:
subs_data = collect_info(parent_dir=par_img_dir,
                        exclusion_list=exclusion_list)

In [16]:
subs_data[90].ses

'001'

In [17]:
# TODO:
#   - Write function that takes input (of array) of SubDataInfo object(s) {sub,data,ses}, and search dict - to search and categorize data
#   - helper/support function to aid in search and categorization of data.

In [10]:
def convert_modality(sub_data: SubDataInfo,
                     out_dir: str,
                     search_dict: Dict[str,str],
                     meta_dict: Optional[Dict[str,Union[int,str]]] = None,
                     keep_unknown: Optional[bool] = False):
    '''working doc-string'''

    mod_found: bool = False

    sub = sub_data.sub
    ses = sub_data.ses
    data = sub_data.data

    if 'dcm' in data:
        if is_valid_dcm(dcm_file=data,raise_exc=False,verbose=False):
            pass
        else:
            return None
    
    for key,item in search_dict.items():
        for dict_key,dict_item in search_dict[key].items():
            if isinstance(dict_item,list):
                if list_in_substr(in_list=dict_item,in_str=data):
                    mod_found = True
                    if verbose:
                        print(f"{key} - {dict_key}: {dict_item}")
                    modality_type = key
                    modality_label = dict_key
                    [com_param_dict, scan_param_dict] = get_metadata(dictionary=meta_dict,modality_type=modality_type)
                    # Conditionals for file conversion
            elif isinstance(dict_item,dict):
                tmp_dict = search_dict[key]
                for d_key,d_item in tmp_dict[dict_key].items():
                    if list_in_substr(in_list=d_item,in_str=data):
                        if verbose:
                            print(f"{key} - {dict_key} - {d_key}: {d_item}")
                        modality_type = key
                        modality_label = dict_key
                        task = d_key
                        [com_param_dict, scan_param_dict] = utils.get_metadata(dictionary=meta_dict,modality_type=modality_type,task=task)
                        # Conditionals for file conversion

In [31]:
# Pass the large number of args as a dict
# 
# Needs to run within for-loop
# 
# Unable to test without actual data
# 
# img_data = convert_image_data(file=subs_data[0].data,
#                                 basename="random_str",
#                                 out_dir="out_dir")

In [25]:
# convert_to_bids
def data_to_bids(sub_data: SubDataInfo,
                 out_dir: str,
                 modality_type: Optional[str] = "",
                 modality_label: Optional[str] = "",
                 task: Optional[str] = "",
                 meta_dict: Optional[Dict] = {},
                 mod_dict: Optional[Dict] = {},
                 log: Optional[LogFile] = None,
                 env: Dict = {},
                 dryrun: bool = False
                 ):
    '''working doc-string'''
    sub_dir = os.path.join(out_dir,"sub-" + sub_data.sub)
    sub_tmp = sub_dir
    
    if sub_data.ses:
        sub_dir = os.path.join(sub_dir,"ses-" + sub_data.ses)
    
    if sub_dir:
        sub_dir: str = os.path.abspath(sub_dir)
    else:
        os.makedirs(sub_dir)
        sub_dir: str = os.path.abspath(sub_dir)
    
    if '.par' in sub_data.data or '.dcm' in sub_data.data:
        with TmpDir(tmp_dir=sub_tmp,use_cwd=False) as tmp:
            with TmpDir.TmpFile(tmp_dir=tmp.tmp_dir) as f:
                tmp.mk_tmp_dir()
                [_path, basename, _ext] = f.file_parts()
                img_data = convert_image_data(file=sub_data.data,
                                              basename=basename,
                                              out_dir=tmp.tmp_dir,
                                              log=log,
                                              env=env,
                                              dryrun=dryrun,
                                              return_obj=True
                                             )
                
                for i in len(img_data.imgs):
                    if img_data.jsons[i]:
                        json_dict: Dict = read_json(json_file=img_data.jsons[i])
                    metadata: Dict = dict_multi_update(**meta_dict,
                                                       **mod_dict)
                    bids_dict: Dict = construct_bids_dict(meta_dict=metadata,
                                                          json_dict=json_dict)
                
                # Put modality specific conversion functions here
    elif '.nii' in sub_data.data:
        # Handle NIFTI cases here
        pass
    else:
        return None

In [ ]:
def proc_batch(s:str,
               search_dict: Dict,
               bids_search: Optional[Dict] = None,
               bids_map: Optional[Dict] = None
              ):
    '''
    TODO: 
        * Use this function to search through image files
        * This function should return lists of image data files
            OR
        * List of subject data (as a bids_name_dict) and the image files.
    '''
    # use this function to iterate through image data files and subjects
    search_arr: List[str] = list_dict(d=search_dict)
    
    for i in search_arr:
        for k,v in i.items():
            if depth(i) == 3:
                for k2,v2 in v.items():
                    modality_type = k
                    modality_label = k2
                    mod_search = v2
                    print(f"{modality_type} - {modality_label} - {mod_search}")
                    # Do stuff here
                    # Search str with mod_search list of substrings
                    print(list_in_substr(in_list=mod_search,in_str=s))
                    if list_in_substr(in_list=mod_search,in_str=s):
                        bids_name_dict = search_bids(s=s,
                                                     bids_search=bids_search,
                                                     bids_map=bids_map,
                                                     modality_type=modality_type,
                                                     modality_label=modality_label)
            elif depth(i) == 4:
                for k2,v2 in v.items():
                    for k3,v3 in v2.items():
                        modality_type = k
                        modality_label = k2
                        task = k3
                        mod_search = v3
                        print(f"{modality_type} - {modality_label} - {task} - {mod_search}")
                        # Do stuff here
                        if list_in_substr(in_list=mod_search,in_str=s):
                            bids_name_dict = search_bids(s=s,
                                                         bids_search=bids_search,
                                                         bids_map=bids_map,
                                                         modality_type=modality_type,
                                                         modality_label=modality_label,
                                                         task=task)
    # return bids_name_dict